# Hospital Readmission Prediction Project
## Phase 6-7: Model Building & Evaluation

**Author:** Vindya Siriwardhana  
**Previous Phase Results:**
- Features: 26 engineered features
- Training: Balanced with SMOTE (1:1)
- Test: Original distribution (11.16% readmission)

---
## SETUP & LOAD PREPARED DATA

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix,
    classification_report, precision_recall_curve
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Libraries imported successfully!")

In [ ]:
# Load prepared data from Phase 4-5
with open('/home/claude/hospital_readmission_prepared_data.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']
feature_names = data['feature_names']

print("✅ Data loaded successfully!")
print(f"\n📊 Training set: {X_train.shape}")
print(f"📊 Test set: {X_test.shape}")
print(f"📊 Features: {len(feature_names)}")
print(f"\n🎯 Training balance: {pd.Series(y_train).value_counts()[1]} : {pd.Series(y_train).value_counts()[0]}")
print(f"🎯 Test readmission rate: {y_test.mean()*100:.2f}%")

---
## PHASE 6: MODEL BUILDING (Steps 16-19)

### Step 16: Train Baseline Model (Logistic Regression)

In [ ]:
print("\n" + "="*80)
print("STEP 16: LOGISTIC REGRESSION (BASELINE MODEL)")
print("="*80)

# Train Logistic Regression
print("\n🔧 Training Logistic Regression...")

lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

print("✅ Model trained!")

# Predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Evaluation
lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_precision = precision_score(y_test, y_pred_lr)
lr_recall = recall_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr)
lr_auc = roc_auc_score(y_test, y_pred_proba_lr)

print("\n📊 LOGISTIC REGRESSION RESULTS:")
print(f"   Accuracy:  {lr_accuracy:.4f}")
print(f"   Precision: {lr_precision:.4f}")
print(f"   Recall:    {lr_recall:.4f}")
print(f"   F1-Score:  {lr_f1:.4f}")
print(f"   AUC-ROC:   {lr_auc:.4f}")

### Step 17: Train Random Forest

In [ ]:
print("\n" + "="*80)
print("STEP 17: RANDOM FOREST")
print("="*80)

# Train Random Forest
print("\n🔧 Training Random Forest...")

rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

print("✅ Model trained!")

# Predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluation
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf)
rf_recall = recall_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)
rf_auc = roc_auc_score(y_test, y_pred_proba_rf)

print("\n📊 RANDOM FOREST RESULTS:")
print(f"   Accuracy:  {rf_accuracy:.4f}")
print(f"   Precision: {rf_precision:.4f}")
print(f"   Recall:    {rf_recall:.4f}")
print(f"   F1-Score:  {rf_f1:.4f}")
print(f"   AUC-ROC:   {rf_auc:.4f}")

### Step 18: Train XGBoost

In [ ]:
print("\n" + "="*80)
print("STEP 18: XGBOOST")
print("="*80)

# Train XGBoost
print("\n🔧 Training XGBoost...")

xgb_model = XGBClassifier(
    n_estimators=100,
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)
xgb_model.fit(X_train, y_train)

print("✅ Model trained!")

# Predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Evaluation
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_precision = precision_score(y_test, y_pred_xgb)
xgb_recall = recall_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb)
xgb_auc = roc_auc_score(y_test, y_pred_proba_xgb)

print("\n📊 XGBOOST RESULTS:")
print(f"   Accuracy:  {xgb_accuracy:.4f}")
print(f"   Precision: {xgb_precision:.4f}")
print(f"   Recall:    {xgb_recall:.4f}")
print(f"   F1-Score:  {xgb_f1:.4f}")
print(f"   AUC-ROC:   {xgb_auc:.4f}")

### Step 19: Hyperparameter Tuning (Best Model)

In [ ]:
print("\n" + "="*80)
print("STEP 19: HYPERPARAMETER TUNING")
print("="*80)

# Compare models to select best for tuning
results_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [lr_accuracy, rf_accuracy, xgb_accuracy],
    'Precision': [lr_precision, rf_precision, xgb_precision],
    'Recall': [lr_recall, rf_recall, xgb_recall],
    'F1-Score': [lr_f1, rf_f1, xgb_f1],
    'AUC-ROC': [lr_auc, rf_auc, xgb_auc]
})

print("\n📊 Model Comparison:")
print(results_df.to_string(index=False))

# Select best model based on AUC-ROC
best_model_idx = results_df['AUC-ROC'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
print(f"\n🏆 Best model by AUC-ROC: {best_model_name}")

In [ ]:
# Hyperparameter tuning for best model (assuming XGBoost or RF)
print(f"\n🔧 Tuning hyperparameters for {best_model_name}...")
print("⏳ This may take 5-10 minutes...")

if 'XGBoost' in best_model_name:
    # XGBoost tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0]
    }
    
    grid_search = GridSearchCV(
        XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False),
        param_grid,
        cv=3,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )
    
elif 'Random Forest' in best_model_name:
    # Random Forest tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42),
        param_grid,
        cv=3,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )
else:
    # Logistic Regression tuning
    param_grid = {
        'C': [0.01, 0.1, 1, 10],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear']
    }
    
    grid_search = GridSearchCV(
        LogisticRegression(random_state=42, max_iter=1000),
        param_grid,
        cv=3,
        scoring='roc_auc',
        n_jobs=-1,
        verbose=1
    )

# Fit grid search
grid_search.fit(X_train, y_train)

print("\n✅ Hyperparameter tuning complete!")
print(f"\n🎯 Best parameters: {grid_search.best_params_}")
print(f"🎯 Best cross-validation AUC-ROC: {grid_search.best_score_:.4f}")

In [ ]:
# Get best model
best_model = grid_search.best_estimator_

# Predictions with tuned model
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

# Evaluation
best_accuracy = accuracy_score(y_test, y_pred_best)
best_precision = precision_score(y_test, y_pred_best)
best_recall = recall_score(y_test, y_pred_best)
best_f1 = f1_score(y_test, y_pred_best)
best_auc = roc_auc_score(y_test, y_pred_proba_best)

print("\n📊 TUNED MODEL RESULTS:")
print(f"   Accuracy:  {best_accuracy:.4f}")
print(f"   Precision: {best_precision:.4f}")
print(f"   Recall:    {best_recall:.4f}")
print(f"   F1-Score:  {best_f1:.4f}")
print(f"   AUC-ROC:   {best_auc:.4f}")

---
## PHASE 7: MODEL EVALUATION (Steps 20-22)

### Step 20: Calculate All Metrics

In [ ]:
print("\n" + "="*80)
print("STEP 20: COMPREHENSIVE METRICS")
print("="*80)

# Final comparison with tuned model
final_results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', f'{best_model_name} (Tuned)'],
    'Accuracy': [lr_accuracy, rf_accuracy, xgb_accuracy, best_accuracy],
    'Precision': [lr_precision, rf_precision, xgb_precision, best_precision],
    'Recall': [lr_recall, rf_recall, xgb_recall, best_recall],
    'F1-Score': [lr_f1, rf_f1, xgb_f1, best_f1],
    'AUC-ROC': [lr_auc, rf_auc, xgb_auc, best_auc]
})

print("\n📊 FINAL MODEL COMPARISON:")
print(final_results.to_string(index=False))

# Highlight best scores
print("\n🏆 BEST SCORES:")
for col in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']:
    best_idx = final_results[col].idxmax()
    best_val = final_results[col].max()
    best_mod = final_results.loc[best_idx, 'Model']
    print(f"   {col:12s}: {best_val:.4f} ({best_mod})")

In [ ]:
# Visualize metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
final_results_plot = final_results.set_index('Model')[metrics_to_plot]

fig, ax = plt.subplots(figsize=(14, 6))
final_results_plot.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Model Performance Comparison', fontsize=16, fontweight='bold')
ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_ylim([0, 1.0])
ax.legend(loc='lower right')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Step 21: Confusion Matrices & ROC Curves

In [ ]:
print("\n" + "="*80)
print("STEP 21: CONFUSION MATRICES & ROC CURVES")
print("="*80)

In [ ]:
# Confusion matrices for all models
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

models_data = [
    ('Logistic Regression', y_pred_lr),
    ('Random Forest', y_pred_rf),
    ('XGBoost', y_pred_xgb),
    (f'{best_model_name} (Tuned)', y_pred_best)
]

for idx, (name, y_pred) in enumerate(models_data):
    row = idx // 2
    col = idx % 2
    
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[row, col],
                xticklabels=['Not Readmit', 'Readmit'],
                yticklabels=['Not Readmit', 'Readmit'])
    axes[row, col].set_title(f'{name}\nConfusion Matrix', fontsize=12, fontweight='bold')
    axes[row, col].set_xlabel('Predicted')
    axes[row, col].set_ylabel('Actual')

plt.tight_layout()
plt.show()

print("\n✅ Confusion matrices plotted!")

In [ ]:
# ROC Curves for all models
plt.figure(figsize=(10, 8))

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_proba_lr)
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC={lr_auc:.4f})', linewidth=2)

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC={rf_auc:.4f})', linewidth=2)

# XGBoost
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC={xgb_auc:.4f})', linewidth=2)

# Tuned model
fpr_best, tpr_best, _ = roc_curve(y_test, y_pred_proba_best)
plt.plot(fpr_best, tpr_best, label=f'{best_model_name} Tuned (AUC={best_auc:.4f})', linewidth=2, linestyle='--')

# Random classifier line
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - All Models', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ ROC curves plotted!")

### Step 22: Select Best Model

In [ ]:
print("\n" + "="*80)
print("STEP 22: SELECT BEST MODEL")
print("="*80)

# Select best model (tuned model)
final_model = best_model

print(f"\n🏆 FINAL SELECTED MODEL: {best_model_name} (Tuned)")
print("\n📊 FINAL MODEL PERFORMANCE:")
print(f"   Accuracy:  {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"   Precision: {best_precision:.4f} ({best_precision*100:.2f}%)")
print(f"   Recall:    {best_recall:.4f} ({best_recall*100:.2f}%)")
print(f"   F1-Score:  {best_f1:.4f}")
print(f"   AUC-ROC:   {best_auc:.4f}")

print("\n💡 INTERPRETATION:")
print(f"   • Model correctly identifies {best_accuracy*100:.1f}% of all patients")
print(f"   • When model predicts readmission, it's correct {best_precision*100:.1f}% of the time")
print(f"   • Model catches {best_recall*100:.1f}% of actual readmissions")
print(f"   • AUC-ROC of {best_auc:.3f} indicates {'excellent' if best_auc > 0.8 else 'good' if best_auc > 0.7 else 'fair'} discrimination")

In [ ]:
# Classification report
print("\n📋 DETAILED CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred_best, 
                          target_names=['Not Readmitted', 'Readmitted <30']))

In [ ]:
# Feature importance (if tree-based model)
if hasattr(final_model, 'feature_importances_'):
    print("\n📊 TOP 10 MOST IMPORTANT FEATURES:")
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': final_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(feature_importance.head(10).to_string(index=False))
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['Importance'])
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Importance', fontsize=12)
    plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Feature importance analyzed!")

---
## 💾 SAVE FINAL MODEL

In [ ]:
print("\n" + "="*80)
print("SAVING FINAL MODEL")
print("="*80)

# Save model and results
model_artifacts = {
    'model': final_model,
    'model_name': best_model_name,
    'feature_names': feature_names,
    'scaler': data['scaler'],
    'label_encoders': data['label_encoders'],
    'metrics': {
        'accuracy': best_accuracy,
        'precision': best_precision,
        'recall': best_recall,
        'f1': best_f1,
        'auc': best_auc
    },
    'best_params': grid_search.best_params_
}

with open('/home/claude/hospital_readmission_final_model.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

print("\n💾 Saved:")
print("   ✓ Final trained model")
print("   ✓ Feature names")
print("   ✓ Scaler & encoders")
print("   ✓ Performance metrics")
print("   ✓ Best hyperparameters")

print("\n📁 File: hospital_readmission_final_model.pkl")
print("\n✅ Model ready for deployment!")

---
## 📊 PHASE 6-7 SUMMARY

In [ ]:
print("\n" + "="*80)
print("PHASE 6-7 COMPLETED SUMMARY")
print("="*80)

print("\n✅ COMPLETED STEPS:")
print("   Step 16: ✓ Logistic Regression (baseline)")
print("   Step 17: ✓ Random Forest")
print("   Step 18: ✓ XGBoost")
print("   Step 19: ✓ Hyperparameter tuning (GridSearchCV)")
print("   Step 20: ✓ Comprehensive metrics calculated")
print("   Step 21: ✓ Confusion matrices & ROC curves")
print("   Step 22: ✓ Best model selected & saved")

print(f"\n🏆 FINAL MODEL: {best_model_name} (Tuned)")
print(f"\n📊 KEY METRICS:")
print(f"   • AUC-ROC:   {best_auc:.4f}")
print(f"   • Accuracy:  {best_accuracy:.4f}")
print(f"   • Precision: {best_precision:.4f}")
print(f"   • Recall:    {best_recall:.4f}")
print(f"   • F1-Score:  {best_f1:.4f}")

print("\n🎯 NEXT STEPS (Phase 8):")
print("   Step 23: Feature importance analysis")
print("   Step 24: SHAP values implementation")
print("   Step 25: SHAP visualizations")

print("\n" + "="*80)
print("Ready to proceed to Phase 8: Explainability (SHAP)!")
print("="*80)